In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

- `ResponseSchema` 클래스를 사용하여 사용자의 질문에 대한 답변과 사용된 소스(웹사이트)에 대한 설명을 포함하는 응답 스키마를 정의합니다.
- `StructuredOutputParser`를 `response_schemas`를 사용하여 초기화하여, 정의된 응답 스키마에 따라 출력을 구조화합니다.


In [3]:
# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [5]:
print(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자의 질문에 대한 답변
	"source": string  // 사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.
}
```


In [6]:
format_instruction = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible. \n{format_instruction}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instruction":format_instruction}
)

In [7]:
model = ChatOpenAI(model='gpt-4o-mini',temperature=0)
chain = prompt | model | output_parser

In [8]:
chain.invoke({"question":"최고의 프로그래밍 언어는?"})

{'answer': "최고의 프로그래밍 언어는 사용자의 필요와 프로젝트의 요구 사항에 따라 다릅니다. 예를 들어, 웹 개발에는 JavaScript가, 데이터 과학에는 Python이, 시스템 프로그래밍에는 C가 많이 사용됩니다. 각 언어는 특정 용도에 최적화되어 있으므로, 어떤 언어가 '최고'인지는 상황에 따라 달라질 수 있습니다.",
 'source': 'https://www.codecademy.com/articles/what-is-the-best-programming-language'}

In [9]:
for s in chain.stream({"question":"최고의 프로그래밍 언어는?"}):
    print(s)

{'answer': "최고의 프로그래밍 언어는 사용자의 필요와 프로젝트의 요구 사항에 따라 다릅니다. 예를 들어, 웹 개발에는 JavaScript가, 데이터 과학에는 Python이, 시스템 프로그래밍에는 C가 많이 사용됩니다. 각 언어는 특정 용도에 최적화되어 있으므로, '최고'라는 개념은 상대적입니다.", 'source': 'https://www.codecademy.com/articles/which-programming-language-should-i-learn'}
